# Part I. Create ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [18]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('Total number of lines:', sum(1 for line in f))

Total number of lines: 6821


# Part II. Create table and query them using Apache Cassandra. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Connect to Apache Cassandra and create a keyspace

#### Creating a Cluster

In [22]:
# Create a connection to a Cassandra instance your local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

print("Connected to the cluster")

Connected to the cluster


#### Create Keyspace

In [21]:
session.execute("""
                CREATE KEYSPACE IF NOT EXISTS sparkify
                WITH REPLICATION =
                {'class':'SimpleStrategy', 'replication_factor':'1'}
                """)

print("Successfully created a keyspace")

Successfully created a keyspace


#### Set Keyspace

In [24]:
session.set_keyspace("sparkify")

### Now we need to create tables to run the following queries. With Apache Cassandra we model the database tables based on the queries we want to run.

## Create queries to ask the following questions about the data

### 1. Query the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Perform a CREATE statement to create the first table

In [8]:
## Query: Create the table "sparkify_by_sessionid" that has sessionId, itemInSession, artist, song and length columns. \
## The sessionId and itemInSession columns are used as the composite partition primary key.

query_1_create = "CREATE TABLE IF NOT EXISTS sparkify_by_sessionid "
query_1_create = query_1_create + "(sessionId int, itemInSession int, artist text, song text, length decimal, PRIMARY KEY ((sessionId, itemInSession)))"

session.execute(query_1_create)

#### Perform INSERT statements to insert data into the first table

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sparkify_by_sessionid (sessionId, itemInSession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Perform a SELECT statement to verify that the data have been inserted into the first table

In [11]:
## Query: Select the artist, song title and song's length that was heard during \
## sessionId = 338, and itemInSession = 4 from the "sparkify_by_sessionid" table.

query_1_select = "SELECT artist, song, length FROM sparkify_by_sessionid WHERE sessionId = 338 AND itemInSession = 4"         

results = session.execute(query_1_select)

for row in results:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### 2. Query the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Perform a CREATE statement to create the second table

In [39]:
## Query: Create the table "sparkify_by_userid" that has userId, sessionId, itemInSession, artist, song, firstName and lastName columns. \
## The userId and sessionId columns are used as the composite partition primary key while itemInSession is used as the clustering column.

query_2_create = "CREATE TABLE IF NOT EXISTS sparkify_by_userid "
query_2_create = query_2_create + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"

session.execute(query_2_create)

#### Perform INSERT statements to insert data into the second table

In [40]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sparkify_by_userid (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Perform a SELECT statement to verify that the data have been inserted into the second table

In [41]:
## Query: Select the artist, song title, user's first name and last name that from \
## userId = 10, and sessionId = 182 from the "sparkify_by_userid" table.

query_2_select = "SELECT artist, song, firstName, lastName FROM sparkify_by_userid WHERE userId = 10 AND sessionId = 182"              

results = session.execute(query_2_select)

for row in results:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Query every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Perform a CREATE statement to create the third table

In [34]:
## Query: Create the table "sparkify_by_song" that has song title, user name (first and last) columns. \
## The song title is used as the partition key and userId as the clustering column.

query_3_create = "CREATE TABLE IF NOT EXISTS sparkify_by_song "
query_3_create = query_3_create + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"

session.execute(query_3_create)

#### Perform INSERT statements to insert data into the third table

In [35]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sparkify_by_song (song, userId, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s) IF NOT EXISTS"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Perform a SELECT statement to verify that the data have been inserted into the third table

In [36]:
## Query: Select every user name (first and last) who listened to the song 'All Hands Against His Own' \
## from the "sparkify_by_song" table.

query_3_select = "SELECT firstName, lastName FROM sparkify_by_song WHERE song = 'All Hands Against His Own'"              

results = session.execute(query_3_select)

for row in results:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [37]:
drop_queries = []

drop_queries.append("DROP TABLE IF EXISTS sparkify_by_sessionid")
drop_queries.append("DROP TABLE IF EXISTS sparkify_by_userid")
drop_queries.append("DROP TABLE IF EXISTS sparkify_by_song")

for query in drop_queries:
    session.execute(query)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()